In [1]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import *

print('SAGA code version', SAGA.__version__)

SAGA code version 0.3.5


In [2]:
# import other modules

from collections import defaultdict
from astropy.table import Table
from astropy.coordinates import SkyCoord
from easyquery import Query

In [3]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')

saga_database_dr12 = SAGA.Database()
saga_database_dr12.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr12_nsa{}.fits.gz'

saga_database_dr14 = SAGA.Database()
saga_database_dr14.base_file_path_pattern = '/home/yymao/Documents/Research/SAGA/new_base_catalogs/base_dr14_nsa{}.fits.gz'

saga_object_catalog = SAGA.ObjectCatalog(saga_database)
saga_object_catalog_dr12 = SAGA.ObjectCatalog(saga_database_dr12)
saga_object_catalog_dr14 = SAGA.ObjectCatalog(saga_database_dr14)

saga_host_catalog = SAGA.HostCatalog(saga_database)

In [4]:
def get_counts(base_list):
    data = defaultdict(list)

    for t in base_list:
        host_id = t['HOST_NSAID'][0]

        data['SAGA name'].append(saga_host_catalog.id_to_name(host_id))
        data['NSAID'].append(host_id)
        data['sat'].append((C.is_sat & C.is_clean & C.has_spec).count(t))
        data['low_z'].append((C.is_low_z & C.is_clean & C.has_spec).count(t))

        for i in (-1, 0, 1, 2, 3, 4):
            data['zq_{}'.format(i)].append(Query('ZQUALITY=={}'.format(i)).count(t))

        for i in (-1, 0, 1, 2, 3, 4, 5):
            data['rm_{}'.format(i)].append(Query('REMOVE=={}'.format(i)).count(t))

    return Table(data)

## load base catalogs

In [5]:
# specify columns to load
columns = 'OBJID HOST_NSAID RA DEC REMOVE SATS SPEC_Z ZQUALITY TELNAME SPEC_REPEAT r_mag coord'.split()
columns.extend(get_sdss_colors()[:3])
columns.extend(map('{}_err'.format, get_sdss_colors()[:3]))

cuts = C.is_galaxy & C.fibermag_r_cut

# load base catalogs (will take a while)
catalogs_orig = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
catalogs_dr12 = saga_object_catalog_dr12.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')
catalogs_dr14 = saga_object_catalog_dr14.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')

## tables of counts

In [6]:
get_counts(catalogs_orig)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,33770,0,109,37,474,2910,34426,0,11,154,1687,116,906
Odyssey,147100,9,14,33868,0,117,53,307,2509,33723,0,4,153,2105,145,724
Dune,165536,1,8,32808,0,171,128,388,3775,34734,0,17,155,1389,141,834
AnaK,61945,3,7,35070,0,166,41,573,2631,35281,0,18,235,1963,139,845
Narnia,132339,2,5,36935,0,146,20,212,1959,35998,0,6,203,2150,104,811
OBrother,149781,4,4,33147,0,91,41,162,1304,31563,0,6,98,2062,47,969
StarTrek,33446,2,5,33578,0,102,8,124,1639,32015,0,9,55,2478,102,792
Catch22,150887,6,11,31694,0,254,144,156,1392,30641,0,19,80,1596,84,1220
ScoobyDoo,161174,4,8,34573,0,37,2,43,1000,33441,0,2,71,1316,63,762


In [7]:
get_counts(catalogs_dr12)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,33803,0,107,33,464,2893,34010,0,21,145,1676,542,906
Odyssey,147100,9,14,33853,0,118,51,297,2536,33253,0,8,150,2081,639,724
Dune,165536,1,7,32764,0,166,120,383,3837,34338,0,42,130,1379,547,834
AnaK,61945,3,6,35075,0,160,40,571,2635,34273,0,49,208,1962,1144,845
Narnia,132339,2,4,36947,0,152,18,202,1953,35107,0,35,175,2147,997,811
OBrother,149781,4,4,33177,0,81,37,157,1293,31292,0,7,97,2061,319,969
StarTrek,33446,2,4,33595,0,99,8,119,1630,31409,0,31,37,2474,708,792
Catch22,150887,6,10,31714,0,250,142,152,1382,30171,0,43,62,1593,551,1220
ScoobyDoo,161174,4,7,34584,0,32,2,42,995,32967,0,19,55,1316,536,762


In [8]:
get_counts(catalogs_dr14)

SAGA name,NSAID,sat,low_z,zq_-1,zq_0,zq_1,zq_2,zq_3,zq_4,rm_-1,rm_0,rm_1,rm_2,rm_3,rm_4,rm_5
str11,int32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
Gilgamesh,166313,2,9,33739,0,105,33,464,2861,33960,0,21,145,1663,511,902
Odyssey,147100,9,14,33834,0,120,51,297,2496,33236,0,8,150,2073,608,723
Dune,165536,1,7,32670,0,162,118,384,3813,34281,0,42,130,1365,500,829
AnaK,61945,3,6,34650,0,259,40,574,2769,34006,0,49,207,1937,1254,839
Narnia,132339,2,4,37051,0,167,19,202,1784,35209,0,35,175,2145,850,809
OBrother,149781,4,4,33147,0,78,37,157,1277,31276,0,7,97,2050,303,963
StarTrek,33446,2,4,33586,0,102,8,119,1616,31399,0,31,37,2472,701,791
Catch22,150887,6,10,31732,0,240,142,152,1372,30187,0,43,62,1592,533,1221
ScoobyDoo,161174,4,7,34836,0,29,2,42,976,33189,0,19,55,1335,517,770


## check satellites

In [13]:
host_idx = -3
is_sat = (C.is_sat & C.is_clean & C.has_spec)

In [14]:
is_sat.filter(catalogs_orig[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int64,float64,float64,int64,int64,float32,int64,str6,str48,float32,object,float32,float32,float32,float32,float32,float32


In [15]:
is_sat.filter(catalogs_dr12[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int32,float64,float64,int16,int16,float32,int16,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237651735757259464,13927,180.637211465,1.96106269885,-1,1,0.00696,4,MMT,MMT,22.1907,"180.63721146497858,1.9610626988481503",-0.336117,0.883909,1.11828,0.956412,0.524856,0.342213


In [16]:
is_sat.filter(catalogs_dr14[host_idx])

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int32,float64,float64,int16,int16,float32,int16,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237651735757259464,13927,180.63721146,1.96106269321,-1,1,0.00696,4,MMT,MMT,22.2069,"180.6372114598704,1.9610626932090294",-0.328957,0.888901,1.11794,0.955371,0.524457,0.342155


In [17]:
find_near_objid(catalogs_orig[host_idx], 1237651735757259464, 20)

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int64,float64,float64,int64,int64,float32,int64,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237651735757259099,13927,180.636494306,1.95802947978,5,1,0.00696,4,MMT,MMT,18.6114,"180.6364943057221,1.9580294797807452",7.64154,0.567709,-0.32235,3.49089,0.123041,0.193003
1237651735757259464,13927,180.637211465,1.96106269885,-1,-1,-1.0,-1,,,22.1907,"180.63721146497858,1.9610626988481503",-0.336117,0.883909,1.11828,0.956412,0.524856,0.342213


In [18]:
find_near_objid(catalogs_dr14[host_idx], 1237651735757259464, 20)

OBJID,HOST_NSAID,RA,DEC,REMOVE,SATS,SPEC_Z,ZQUALITY,TELNAME,SPEC_REPEAT,r_mag,coord,ug,gr,ri,ug_err,gr_err,ri_err
,,,,,,,,,,,"deg,deg",,,,,,
int64,int32,float64,float64,int16,int16,float32,int16,str6,str48,float32,object,float32,float32,float32,float32,float32,float32
1237651735757259464,13927,180.63721146,1.96106269321,-1,1,0.00696,4,MMT,MMT,22.2069,"180.6372114598704,1.9610626932090294",-0.328957,0.888901,1.11794,0.955371,0.524457,0.342155
1237651735757259099,13927,180.636494302,1.95802947582,5,-1,-1.0,-1,,,18.6277,"180.6364943021375,1.9580294758235546",7.6367,0.573706,-0.322073,3.49035,0.123041,0.193003
